In [ ]:
---
title: Resonance of spring-mass-damper system
description: Poles and their effects
author: Daning H.
show-code: False
show-prompt: False
params:
    w0:
        input: numeric
        label: Natural frequency, omega_0
        value: 2.0
        step: 0.5
    zt:
        input: numeric
        label: Damping ratio, zeta
        value: 0.0
        step: 0.1
    decay:
        input: numeric
        label: Decay rate of the input, sigma
        value: 0.0
        step: 1.0
    extra:
        input: checkbox
        label: Gain plot
        value: False
---

From previous courses you should be familiar with the concept of resonance.  For a spring-mass-damper system, when the excitation frequency is closer to the system frequency, the system response becomes maximized and unbounded if there is no damping.

Laplace Transform gives us a new way to look at resonance, via the concept of \textit{poles}.  Specifically, if we write the governing equation,
$$
y'' + 2\zeta\omega_0 y' + \omega_0^2 y = \exp(-\sigma t)\cos(\omega t)
$$
where $\omega_0$ is the natural frequency, $\zeta$ is damping ratio, $y$ is the displacement, and the right-hand side is an input with decay rate $\sigma$ and frequency $\omega$.  The transfer function is,
$$
Q(s) = \frac{1}{s^2 + 2\zeta\omega_0 s + \omega_0^2}
$$
The Laplace Transform of the input is
$$
R(s) = \frac{s+\sigma}{(s+\sigma)^2+\omega^2}
$$

While looking at the interaction, let's fix $\omega_0=2$ and sweep over different $\omega$.

+ First, let $\zeta=0$ and $\sigma=0$.  Locate the roots of the denominators of $Q(s)$ and $R(s)$ on the right figure, which shows the $s$-plane.  The slider controls the value of $\omega$, and the time domain response is shown on the left figure.  This is standard resonance, so we know when $\omega=\omega_0$, the amplitude of response becomes maximized (actually unbounded).  On the $s$-plane, we see that resonance corresponds to when the roots (crosses and stars) overlap.

+ Next, let $\zeta=0.2$ and $\sigma=0$.  Same drill, find the roots.  Then, as you change the input frequency, when is the response maximized?  How does the response amplitude correlate to the relative locations between the two sets of roots?  How would your answer change with different values of $\zeta$?

+ Lastly, let $\zeta=0.2$ and $\sigma=0.1$.  After locating the roots, first guess at what $\omega$ will the input produce the maximum amplitude in response, and then verify your guess.  How would your answer change with different values of $\sigma$?

The roots of the denominator of $Q(s)$ are called poles, and determine the dynamic characteristics of the system.  With the knowledge of the poles, one can infer the system behavior without actually finding the time-domain solution - this gives great convenience in the dynamical analysis of engineering systems (e.g., controller design of aircraft).

Two more things to explore if you are really interested.

+ Turn on the Gain plot.  We plot the contours of absolute value of $Q(s)$, or more accurately, the log of $|Q(s)|$ due to the rapid change in $|Q(s)|$.  How is $|Q(s)|$ related to the amplitude of response?  The answer would explain why $|Q(s)|$ is called gain.

+ With all the above knowledge, explore the overdamped case, i.e., $\zeta>1$.

In [2]:
w0 = 2.0
zt = 0.5
decay = -0
extra = True

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from scipy import integrate as spi
from scipy import optimize as spo

decay = -np.abs(decay)

# Generate data
def genA(ab):
    a, b = ab
    A = np.array([[0., 1.], [-b, -a]])
    return A

def genR(k):
    return lambda t, _k=k: np.exp(_k[0]*t) * np.cos(_k[1]*t)

def dyn(t, z, ab, inp):
    A = genA(ab)
    f = np.array([0., inp(t)])
    dz = A.dot(z) + f
    return dz

def gain(S, ab):
    a, b = ab
    G = -np.log(np.abs(S**2 + a*S + b))
    return G

# ODE parameters
a  = 2*zt*w0
b  = w0**2
if zt > 1:
    m, n = -zt*w0, np.sqrt(zt**2-1)*w0
    l1, l2 = m+n, m-n
else:
    re, im = -zt*w0, np.sqrt(1-zt**2)*w0
    l1, l2 = re+1j*im, re-1j*im

# Simulation setup
tf = 16.0
Nt = 201
Ns = 41
ts = np.linspace(0, tf, Nt)
ab = (a, b)
y0 = [0.0, 0.0]

# Max gain location
sol = spo.minimize_scalar(lambda w: -gain(decay+1j*w, ab), bracket=[-4, 4])
wp = np.abs(sol.x)
Nr = gain(np.array(decay+1j*wp), ab)

# t-domain results
inp = []
res = []
ss = np.linspace(0., 4, Ns)
for _s in ss:
    rt  = genR([decay, _s])
    sol = spi.solve_ivp(dyn, [0,tf], y0, t_eval=ts, args=(ab, rt))
    inp.append(rt(ts))
    res.append(sol.y[0])

sol = spi.solve_ivp(dyn, [0,tf], y0, t_eval=ts, args=(ab, genR([decay, wp])))
ref = sol.y[0]

fig = make_subplots(rows=2, cols=2,
                    specs=[[{'rowspan': 1, 'colspan': 1}, {'rowspan': 2, 'colspan': 1}],
                           [{'rowspan': 1, 'colspan': 1}, None]],
                    horizontal_spacing=0.1,
                    subplot_titles=("Time domain input", "s-domain", "Time domain response"))

# s-domain results
N = 41
x = np.linspace(-3, 0.1, N)
y = np.linspace(-4, 4, 2*N)
X, Y = np.meshgrid(x, y)
S = X+1j*Y
N = gain(S, ab)

# Variable plots
## t-domain
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=inp[_i]),
        row=1, col=1)
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='lines', showlegend=False,
        line=dict(color="black", width=1),
        x=ts, y=res[_i]),
        row=2, col=1)

## s-domain
for _i in range(Ns):
    fig.add_trace(go.Scatter(
        visible=False, mode='markers', showlegend=False,
        marker=dict(color="black", symbol="star", size=12),
        x=[decay,decay], y=[-ss[_i],ss[_i]]),
        row=1, col=2)

# Fixed plots
fig.add_trace(go.Scatter(
    visible=True, mode='markers', showlegend=False,
    marker=dict(color="black", symbol="x", size=12),
    x=[l1.real,l2.real], y=[l1.imag,l2.imag]),
    row=1, col=2)

# Extra plots
if extra:
    ## t-domain
    fig.add_trace(go.Scatter(
        visible=True, mode='lines', showlegend=False,
        line=dict(color="rgba(255,0,0,0.5)", width=1),
        x=ts, y=ref),
        row=2, col=1)

    ## s-domain
    fig.add_trace(go.Contour(
        x=x, y=y, z=N, ncontours=16, showscale=False,
        contours_coloring='lines', contours_showlabels=True, line_width=2),
        row=1, col=2)
    fig.add_trace(go.Contour(
        x=x, y=y, z=N, ncontours=10, showscale=False,
        colorscale=[[0, 'rgb(0,0,0)'], [1, 'rgb(0,0,0)']],
        line=dict(width=2, dash="dash"),
        contours=dict(coloring='lines', start=Nr, end=Nr)),
        row=1, col=2)
    fig.add_trace(go.Scatter(
        visible=True, mode='markers', showlegend=False,
        marker=dict(color="rgba(255,0,0,0.5)", symbol="star", size=12),
        x=[decay,decay], y=[-wp,wp]),
        row=1, col=2)

## Initial visibility
fig.data[0].visible = True
fig.data[Ns].visible = True
fig.data[2*Ns].visible = True

# Create and add slider
steps = []
for _i in range(Ns):
    mask = [False] * (Ns*3)
    mask[_i] = True
    mask[_i+Ns] = True
    mask[_i+2*Ns] = True
    mask += [True]
    if extra:
        mask += [True]*4
    step = dict(
        method="update",
        args=[{"visible": mask}],
        label=f"{ss[_i]:3.2f}"
    )
    steps.append(step)

sliders = [dict(
    active=0,
    currentvalue={"prefix": "omega = "},
    pad={"t": 50},
    steps=steps
)]

fig.update_xaxes(title_text='t', row=1, col=1)
fig.update_yaxes(title_text='r(t)', range=[-1.05,1.05], row=1, col=1)
fig.update_xaxes(title_text='t', row=2, col=1)
fig.update_yaxes(title_text='y(t)', range=[np.min(res), np.max(res)], row=2, col=1)
fig.update_xaxes(title_text='Im', range=[-3,0.1], row=1, col=2)
fig.update_yaxes(title_text='Re', range=[-4,4], row=1, col=2)

fig.update_layout(
    sliders=sliders,
    autosize=False,
    width=1000,
    height=600)

fig.show()